In [1]:
import os 
3
os.getcwd() # check your current working diretory
4
os.chdir("C:/Python_DS_Challenges/DataPipeline")

In [2]:
os.getcwd()

'C:\\Python_DS_Challenges\\DataPipeline'

# Create tf.data.Dataset

# There are two ways to createa tf.datast.Dataset

### 1. Data Source

Construct a Dataset from data stored in memory or in files

### 2. Data Transformation
Construct Datasetfrom one or more data objects

# Objective:
    
-   Load data from a CSV
-  Load from numpy and pandas
- Save and load data
       - tf.example
       - TFRecord
- Load image data
- End Result
- Get data in and out of TensorFlow


# Load data from CSV

Import libraries, numpy, pandas and tensorflow

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf

##### Load two CSVs 1. Train, 2. Evaluation csv.  Both are coming from URL

##### Use the Keras Utility from the utils get file as the train.csv and pass in the TRAIN_DATA_URL
What this does is download the CSV directly from the web and then it will put it in the place train filepath and test file path
This is actually the location of the data

In [8]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file('C:\\Python_DS_Challenges\\DataPipeline\\train.csv', TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file('C:\\Python_DS_Challenges\\DataPipeline\\eval.csv', TEST_DATA_URL)

16384/13049 [=====================================] - 0s 0us/step


In [11]:
# See the location of the data

print("The location of the train data." , train_file_path)
print("The location of the test data." , test_file_path)

The location of the train data. C:\Python_DS_Challenges\DataPipeline\train.csv
The location of the test data. C:\Python_DS_Challenges\DataPipeline\eval.csv


# Load the data

 - Specify the features we want to use
   - in this data, we want to use the column called " Survived"


In [12]:
LABEL_COLUMN = 'survived'

In [16]:
def get_dataset(file_path, **kwargs):
    dataset =tf.data.experimental.make_csv_dataset(
        file_path, 
        batch_size=5,
        label_name=LABEL_COLUMN,
        na_value='?',
        num_epochs=1,
        ignore_errors=True,
        **kwargs)
    return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)


In [18]:
raw_test_data

<PrefetchDataset shapes: (OrderedDict([(sex, (None,)), (age, (None,)), (n_siblings_spouses, (None,)), (parch, (None,)), (fare, (None,)), (class, (None,)), (deck, (None,)), (embark_town, (None,)), (alone, (None,))]), (None,)), types: (OrderedDict([(sex, tf.string), (age, tf.float32), (n_siblings_spouses, tf.int32), (parch, tf.int32), (fare, tf.float32), (class, tf.string), (deck, tf.string), (embark_town, tf.string), (alone, tf.string)]), tf.int32)>

In [17]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

 Each item in the dataset is a batch, represented as a tuple of (many examples, many labels). The data from the examples is organized in column-based tensors (rather than row-based tensors), each with as many elements as the batch size (5 in this case).

In [19]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'male' b'male' b'male']
age                 : [28. 36. 26. 11. 42.]
n_siblings_spouses  : [0 0 2 0 1]
parch               : [0 0 0 0 0]
fare                : [56.4958 10.5     8.6625 18.7875 27.    ]
class               : [b'Third' b'Second' b'Third' b'Third' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Cherbourg' b'Southampton']
alone               : [b'y' b'y' b'n' b'y' b'n']


As you can see, the columns in the CSV are named. The dataset constructor will pick these names up automatically. If the file you are working with does not contain the column names in the first line, pass them in a list of strings to the column_names argument in the make_csv_dataset function.



In [20]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'male' b'female' b'female' b'female' b'male']
age                 : [18. 58. 63. 51. 29.]
n_siblings_spouses  : [0 0 0 1 1]
parch               : [0 1 0 0 0]
fare                : [ 13.     153.4625   9.5875  77.9583   7.0458]
class               : [b'Second' b'First' b'Third' b'First' b'Third']
deck                : [b'unknown' b'C' b'unknown' b'D' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'n' b'y' b'n' b'n']


 This example is going to use all the available columns. If you need to omit some columns from the dataset, create a list of just the columns you plan to use, and pass it into the (optional) select_columns argument of the constructor.

In [21]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [30. 28. 16. 25. 28.]
n_siblings_spouses  : [0 8 0 1 0]
class               : [b'First' b'Third' b'Third' b'Third' b'Third']
deck                : [b'E' b'unknown' b'unknown' b'unknown' b'unknown']
alone               : [b'y' b'n' b'y' b'n' b'y']


# Data preprocessing

A CSV file can contain a variety of data types. Typically you want to convert from those mixed types to a fixed length vector before feeding the data into your model.

TensorFlow has a built-in system for describing common input conversions: tf.feature_column, see this tutorial for details.

You can preprocess your data using any tool you like (like nltk or sklearn), and just pass the processed output to TensorFlow.

The primary advantage of doing the preprocessing inside your model is that when you export the model it includes the preprocessing. This way you can pass the raw data directly to your model.

### Continuous data

 If your data is already in an appropriate numeric format, you can pack the data into a vector before passing it off to the model:

In [22]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [28. 28.  4. 28. 49.]
n_siblings_spouses  : [0. 1. 1. 0. 1.]
parch               : [0. 0. 1. 0. 1.]
fare                : [  8.05    15.5     23.       7.8958 110.8833]


In [23]:
example_batch, labels_batch = next(iter(temp_dataset)) 

 Here's a simple function that will pack together all the columns:

In [24]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

In [25]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[35.      0.      0.     10.5   ]
 [24.      0.      2.     14.5   ]
 [20.      0.      0.      9.225 ]
 [28.      0.      0.     26.55  ]
 [27.      0.      0.      7.8958]]

[0 1 0 1 0]


 If you have mixed datatypes you may want to separate out these simple-numeric fields. The tf.feature_column api can handle them, but this incurs some overhead and should be avoided unless really necessary. Switch back to the mixed dataset:

In [26]:
show_batch(raw_train_data)

sex                 : [b'female' b'female' b'female' b'male' b'male']
age                 : [24. 14. 30. 28. 28.]
n_siblings_spouses  : [2 1 0 0 0]
parch               : [3 2 0 0 0]
fare                : [ 18.75   120.      12.475    7.8958   7.7292]
class               : [b'Second' b'First' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'B' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Cherbourg' b'Queenstown']
alone               : [b'n' b'n' b'y' b'y' b'y']


In [27]:
example_batch, labels_batch = next(iter(temp_dataset)) 

 So define a more general preprocessor that selects a list of numeric features and packs them into a single column:

In [28]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [29]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [30]:
show_batch(packed_train_data)

sex                 : [b'female' b'female' b'male' b'female' b'female']
class               : [b'First' b'First' b'Third' b'First' b'Second']
deck                : [b'B' b'C' b'unknown' b'C' b'E']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton' b'Queenstown']
alone               : [b'y' b'n' b'y' b'n' b'y']
numeric             : [[ 29.       0.       0.     211.3375]
 [ 25.       1.       2.     151.55  ]
 [ 19.       0.       0.       8.1583]
 [ 31.       0.       2.     164.8667]
 [ 28.       0.       0.      12.35  ]]


In [31]:
example_batch, labels_batch = next(iter(packed_train_data)) 

### Data Normalization

 Continuous data should always be normalized.

In [33]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc



,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [34]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

# def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

 Now create a numeric column. The tf.feature_columns.numeric_column API accepts a normalizer_fn argument, which will be run on each batch.

Bind the MEAN and STD to the normalizer fn using functools.partial.

In [37]:
# See what you just created.
import functools
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x00000150C4D96318>, mean=array([29.63130781,  0.54545455,  0.37958533, 34.38539856]), std=array([12.51181763,  1.1510896 ,  0.79299921, 54.5977305 ])))

 When you train the model, include this feature column to select and center this block of numeric data:

In [38]:
example_batch['numeric']

<tf.Tensor: id=874, shape=(5, 4), dtype=float32, numpy=
array([[21.    ,  0.    ,  0.    , 77.9583],
       [11.    ,  4.    ,  2.    , 31.275 ],
       [28.    ,  1.    ,  0.    ,  7.75  ],
       [46.    ,  1.    ,  0.    , 61.175 ],
       [28.    ,  0.    ,  0.    ,  8.05  ]], dtype=float32)>

In [39]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.6898524 , -0.47385937, -0.4786705 ,  0.7980716 ],
       [-1.4890968 ,  3.0011094 ,  2.0434003 , -0.05696939],
       [-0.13038135,  0.39488277, -0.4786705 , -0.4878481 ],
       [ 1.3082585 ,  0.39488277, -0.4786705 ,  0.49067244],
       [-0.13038135, -0.47385937, -0.4786705 , -0.4823534 ]],
      dtype=float32)

The mean based normalization used here requires knowing the means of each column ahead of time.

### Categorical data

 Some of the columns in the CSV data are categorical columns. That is, the content should be one of a limited set of 
options.


Use the tf.feature_column API to create a collection with a tf.feature_column.indicator_column for each categorical column.

In [42]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}


In [43]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [44]:
# See what you just created.
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [45]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


This will be become part of a data processing input later when you build the model.

### Combined preprocessing layer

Add the two feature column collections and pass them to a tf.keras.layers.DenseFeatures to create an input layer that will extract and preprocess both input types:

In [47]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [48]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.          0.          1.          0.          0.          0.
  0.          0.          1.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
 -0.6898524  -0.47385937 -0.4786705   0.7980716   0.          1.        ]


### Build the model

 Build a tf.keras.Sequential, starting with the preprocessing_layer.

In [49]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

### Train, evaluate, and predict

 Now the model can be instantiated and trained.

In [51]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [52]:
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 5s 41ms/step - loss: 0.5074 - accuracy: 0.7257
Epoch 2/20
126/126 [==============================] - 1s 5ms/step - loss: 0.4314 - accuracy: 0.8022
Epoch 3/20
126/126 [==============================] - 1s 5ms/step - loss: 0.4021 - accuracy: 0.8166
Epoch 4/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3930 - accuracy: 0.8389
Epoch 5/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3836 - accuracy: 0.8309
Epoch 6/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3678 - accuracy: 0.8373
Epoch 7/20
126/126 [==============================] - 1s 4ms/step - loss: 0.3582 - accuracy: 0.8453
Epoch 8/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3557 - accuracy: 0.8437
Epoch 9/20
126/126 [==============================] - 1s 4ms/step - loss: 0.3470 - accuracy: 0.8453
Epoch 10/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3465 - accuracy: 0.846

 Once the model is trained, you can check its accuracy on the test_data set.

In [53]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

53/53 [==============================] - 1s 15ms/step - loss: 0.4617 - accuracy: 0.8447


Test Loss 0.4616721570491791, Test Accuracy 0.8446969985961914


 Use tf.keras.Model.predict to infer labels on a batch or a dataset of batches.

In [56]:
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))


Predicted survival: 6.39%  | Actual outcome:  SURVIVED
Predicted survival: 99.54%  | Actual outcome:  DIED
Predicted survival: 9.97%  | Actual outcome:  DIED
Predicted survival: 24.69%  | Actual outcome:  SURVIVED
Predicted survival: 92.52%  | Actual outcome:  SURVIVED


# Load NumPy data

load data from NumPy arrays into a tf.data.Dataset.
This example loads the MNIST dataset from a .npz file. However, the source of the NumPy arrays is not important.

In [58]:
import numpy as np
import tensorflow as tf

### Load from .npz file

In [59]:
DATA_URL = 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz'

path = tf.keras.utils.get_file('mnist.npz', DATA_URL)
with np.load(path) as data:
  train_examples = data['x_train']
  train_labels = data['y_train']
  test_examples = data['x_test']
  test_labels = data['y_test']

11493376/11490434 [==============================] - 2s 0us/step


### Load NumPy arrays with tf.data.Dataset

Assuming you have an array of examples and a corresponding array of labels, pass the two arrays as a tuple into tf.data.Dataset.from_tensor_slices to create a tf.data.Dataset.

In [60]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_examples, test_labels))

### Use the datasets

### Shuffle and batch the datasets

In [61]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

### Build and train a model

In [62]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['sparse_categorical_accuracy'])


In [63]:
model.fit(train_dataset, epochs=10)

Epoch 1/10
938/938 [==============================] - 8s 9ms/step - loss: 3.4820 - sparse_categorical_accuracy: 0.87612.67 - 2s 15ms/step - los - 3s 13ms/s - 3s 12ms/step - loss:  - 4s 11ms/step - loss: 6.6921 - sparse_categorica
Epoch 2/10
938/938 [==============================] - 5s 5ms/step - loss: 0.5268 - sparse_categorical_accuracy: 0.9254
Epoch 3/10
938/938 [==============================] - 4s 5ms/step - loss: 0.3851 - sparse_categorical_accuracy: 0.9453
Epoch 4/10
938/938 [==============================] - 5s 5ms/step - loss: 0.3262 - sparse_categorical_accuracy: 0.9539
Epoch 5/10
938/938 [==============================] - 5s 5ms/step - loss: 0.2833 - sparse_categorical_accuracy: 0.9605
Epoch 6/10
938/938 [==============================] - 6s 7ms/step - loss: 0.2626 - sparse_categorical_accuracy: 0.9648
Epoch 7/10
938/938 [==============================] - 5s 6ms/step - loss: 0.2327 - sparse_categorical_accuracy: 0.9687
Epoch 8/10
938/938 [==============================] - 5s

# Load a pandas.DataFrame

use a small dataset provided by the Cleveland Clinic Foundation for Heart Disease. There are several hundred rows in the CSV. Each row describes a patient, and each column describes an attribute. We will use this information to predict whether a patient has heart disease, which in this dataset is a binary classification task.

### Read data using pandas

In [65]:
import pandas as pd
import tensorflow as tf


Download the csv file containing the heart dataset.

In [66]:
csv_file = tf.keras.utils.get_file('heart.csv', 'https://storage.googleapis.com/applied-dl/heart.csv')

16384/13273 [=====================================] - 0s 0us/step


In [67]:
df = pd.read_csv(csv_file)

In [68]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [69]:
df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal         object
target        int64
dtype: object

Convert thal column which is an object in the dataframe to a discrete numerical value.

In [70]:
df['thal'] = pd.Categorical(df['thal'])
df['thal'] = df.thal.cat.codes

In [71]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,2,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,4,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0


### Load data using tf.data.Dataset

# Use tf.data.Dataset.from_tensor_slices to read the values from a pandas dataframe.

One of the advantages of using tf.data.Dataset is it allows you to write simple, highly efficient data pipelines. Read the loading data guide to find out more.

In [72]:
target = df.pop('target')

In [73]:
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))

In [74]:
for feat, targ in dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [ 63.    1.    1.  145.  233.    1.    2.  150.    0.    2.3   3.    0.
   2. ], Target: 0
Features: [ 67.    1.    4.  160.  286.    0.    2.  108.    1.    1.5   2.    3.
   3. ], Target: 1
Features: [ 67.    1.    4.  120.  229.    0.    2.  129.    1.    2.6   2.    2.
   4. ], Target: 0
Features: [ 37.    1.    3.  130.  250.    0.    0.  187.    0.    3.5   3.    0.
   3. ], Target: 0
Features: [ 41.    0.    2.  130.  204.    0.    2.  172.    0.    1.4   1.    0.
   3. ], Target: 0


 Since a pd.Series implements the __array__ protocol it can be used transparently nearly anywhere you would use a np.array or a tf.Tensor.

In [75]:
tf.constant(df['thal'])

<tf.Tensor: id=40426, shape=(303,), dtype=int32, numpy=
array([2, 3, 4, 3, 3, 3, 3, 3, 4, 4, 2, 3, 2, 4, 4, 3, 4, 3, 3, 3, 3, 3,
       3, 4, 4, 3, 3, 3, 3, 4, 3, 4, 3, 4, 3, 3, 4, 2, 4, 3, 4, 3, 4, 4,
       2, 3, 3, 4, 3, 3, 4, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 4,
       4, 2, 3, 3, 4, 3, 4, 3, 3, 4, 4, 3, 3, 4, 4, 3, 3, 3, 3, 4, 4, 4,
       3, 3, 4, 3, 4, 4, 3, 4, 3, 3, 3, 4, 3, 4, 4, 3, 3, 4, 4, 4, 4, 4,
       3, 3, 3, 3, 4, 3, 4, 3, 4, 4, 3, 3, 2, 4, 4, 2, 3, 3, 4, 4, 3, 4,
       3, 3, 4, 2, 4, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4,
       4, 3, 3, 3, 4, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 3, 2,
       4, 4, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 2, 2, 4, 3, 4, 2, 4, 3,
       3, 4, 3, 3, 3, 3, 4, 3, 4, 3, 4, 2, 2, 4, 3, 4, 3, 2, 4, 3, 3, 2,
       4, 4, 4, 4, 3, 0, 3, 3, 3, 3, 1, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 4,
       3, 3, 4, 4, 4, 4, 3, 3, 4, 3, 4, 3, 4, 4, 3, 4, 4, 3, 4, 4, 3

In [76]:
train_dataset = dataset.shuffle(len(df)).batch(1)

# Create and train a model

In [77]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [78]:

model = get_compiled_model()
model.fit(train_dataset, epochs=15)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/15
303/303 [==============================] - 3s 9ms/step - loss: 2.3332 - accuracy: 0.6733876 - 
Epoch 2/15
303/303 [==============================] - 1s 3ms/step - loss: 1.1261 - accuracy: 0.6931
Epoch 3/15
303/303 [==============================] - 1s 2ms/step - loss: 0.9030 - accuracy: 0.6832
Epoch 4/15
303/303 [==============================] - 1s 3ms/step - loss: 0.7757 - accuracy: 0.6931
Epoch 5/15
303/303 [==============================] - 1s 3ms/step - loss: 0.6175 - accuracy: 0.7195
Ep

### Alternative to feature columns

Passing a dictionary as an input to a model is as easy as creating a matching dictionary of tf.keras.layers.Input layers, applying any pre-processing and stacking them up using the functional api. You can use this as an alternative to feature columns.

In [79]:
inputs = {key: tf.keras.layers.Input(shape=(), name=key) for key in df.keys()}
x = tf.stack(list(inputs.values()), axis=-1)

x = tf.keras.layers.Dense(10, activation='relu')(x)
output = tf.keras.layers.Dense(1)(x)

model_func = tf.keras.Model(inputs=inputs, outputs=output)

model_func.compile(optimizer='adam',
                   loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=['accuracy'])

 The easiest way to preserve the column structure of a pd.DataFrame when used with tf.data is to convert the pd.DataFrame to a dict, and slice that dictionary.

In [80]:
dict_slices = tf.data.Dataset.from_tensor_slices((df.to_dict('list'), target.values)).batch(16)


In [81]:
for dict_slice in dict_slices.take(1):
  print (dict_slice)

({'age': <tf.Tensor: id=55148, shape=(16,), dtype=int32, numpy=array([63, 67, 67, 37, 41, 56, 62, 57, 63, 53, 57, 56, 56, 44, 52, 57])>, 'sex': <tf.Tensor: id=55156, shape=(16,), dtype=int32, numpy=array([1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1])>, 'cp': <tf.Tensor: id=55151, shape=(16,), dtype=int32, numpy=array([1, 4, 4, 3, 2, 2, 4, 4, 4, 4, 4, 2, 3, 2, 3, 3])>, 'trestbps': <tf.Tensor: id=55160, shape=(16,), dtype=int32, numpy=
array([145, 160, 120, 130, 130, 120, 140, 120, 130, 140, 140, 140, 130,
       120, 172, 150])>, 'chol': <tf.Tensor: id=55150, shape=(16,), dtype=int32, numpy=
array([233, 286, 229, 250, 204, 236, 268, 354, 254, 203, 192, 294, 256,
       263, 199, 168])>, 'fbs': <tf.Tensor: id=55153, shape=(16,), dtype=int32, numpy=array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0])>, 'restecg': <tf.Tensor: id=55155, shape=(16,), dtype=int32, numpy=array([2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0, 0])>, 'thalach': <tf.Tensor: id=55159, shape=(16,), dtype=int32, n

In [82]:
model_func.fit(dict_slices, epochs=15)

Epoch 1/15
19/19 [==============================] - 1s 51ms/step - loss: 5.2082 - accuracy: 0.4290
Epoch 2/15
19/19 [==============================] - 0s 4ms/step - loss: 3.5295 - accuracy: 0.5875
Epoch 3/15
19/19 [==============================] - 0s 9ms/step - loss: 2.5101 - accuracy: 0.5446
Epoch 4/15
19/19 [==============================] - 0s 5ms/step - loss: 1.7102 - accuracy: 0.6073
Epoch 5/15
19/19 [==============================] - 0s 7ms/step - loss: 1.2429 - accuracy: 0.6502
Epoch 6/15
19/19 [==============================] - 0s 6ms/step - loss: 0.9969 - accuracy: 0.6997
Epoch 7/15
19/19 [==============================] - 0s 7ms/step - loss: 0.8739 - accuracy: 0.7162
Epoch 8/15
19/19 [==============================] - 0s 7ms/step - loss: 0.8051 - accuracy: 0.7228
Epoch 9/15
19/19 [==============================] - 0s 8ms/step - loss: 0.7579 - accuracy: 0.7393
Epoch 10/15
19/19 [==============================] - 0s 6ms/step - loss: 0.7228 - accuracy: 0.7459
Epoch 11/15
19/19 